
General intuition of the code:
- we are simulating a world with agents on a network who choose between EV and gas cars; their decisions depend on (1) what their neighbours do and (2) how much charging infrastructure exists

Plotting functions

# Ratio sweep

One ratio & outcome; reveals tipping points; You vary the initial ratio of EV adopters (or infrastructure level) while keeping other parameters fixed.

You’ve found bistability if:

For some mid-range values, some runs end low and some end high

The results are not a smooth curve but split into two clusters

📌 Two separate plateaus = bistability.
- phase plot: X0 & ration; reveals tipping points and bistability
- spaghetti plot: many simulations; shows how different runs behave over time; reveals multiple stable states (some runs go to 0 some to 1)
- density plot:  similar to spaghetti but smoothed; shows where trajectories cluser

You sweep X₀, I₀, and βᵢ. You uncover:
- critical tipping points
- the size of the bistable region
- how hard (or easy) diffusion is
- how infrastructure feedback matters

From this you conclude: “The system naturally gets stuck unless certain conditions are met.”
This motivates why policy is needed.

Then you rerun the sweeps for different network types. You learn:
- Some networks are more resilient
- Some spread adoption better
- BA networks may need lower X₀
- Random networks may need higher initial adoption

This tells you: “Policy needs to be targeted differently depending on the social structure.”
For example: “Target high-degree hubs first in a BA network.”

Then for the policy design we use the insights from the first 2 parts.

# STEP: Document results and write the report (what to include and where)

For each required part include:
Methods (short):
- Which functions you ran (phase_sweep_df, ratio_sweep_df, collect_intervention_trials)
- Parameter ranges used (list exact ranges)
- Number of trials / batch_size

Results (figures + numbers):
- Phase plot(s) with short caption: what axis means, where tipping boundary is, where bistability shows.
- Ratio sweep plots for representative X₀ values — mark where X* jumps.
- Spaghetti/density/fanchart for baseline vs policy (show mean final X*, quantiles, histograms).
- A short table with numeric summaries: final mean X* (baseline) vs final mean X* (policy), and percent of runs that ended > 0.8.

Policy Interpretation:
- Why the policy works in terms of the model mechanics (moves you across tipping boundary; boosts infrastructure feedback; seeds hubs).
- Practical limitations and assumptions (modelled agents are stylized; network types simplified).

Limitations: model simplifications, choice of parameter ranges, finite trial counts, sensitivity to tau, etc.

# FINAL / reproducibility checklist (what to commit to Git)

Code/ev_core.py (unchanged)

Code/ev_experiments.py (unchanged)

Code/ev_plotting.py (unchanged)

Code/my_assignment.ipynb (your notebook with the cells above + comments)

README.md — short “run me” instructions: pip install -r requirements.txt, then jupyter notebook and run Code/my_assignment.ipynb or python Code/ev_experiments.py for demo.

plots/ — include final .png images referenced in report

report/ — the PDF report and presentation PDF

In [1]:
import numpy as np
import pandas as pd

from ev_experiments import (
    run_timeseries_trial,
    ratio_sweep_df,
    phase_sweep_df,
    traces_to_long_df,
    collect_intervention_trials
)

from ev_plotting import (
    plot_ratio_sweep,
    plot_phase_plot,
    plot_spaghetti,
    plot_density
)


# Baseline scenario dictionary
Goal: run sweeps for X₀, I₀, and βᵢ; detect tipping points & bistability.

Use phase_sweep_df (2D X₀ × ratio heatmap) and ratio_sweep_df (1D ratio → X*). Use spaghetti/density selectively to show bistability at specific parameter settings.

In [2]:
scenario_base = dict( # creates a dictionary of parameters we can pass to sweep fuctions
    I0=0.05, # infrastructure level 
    beta_I=2.0, # how strongly infrastructure increases the coordination payoff a_i = a0 + beta_I * I
    b=1.0, # baselnie payoff for D (ICE); used to compute a_i/b
    g_I=0.05, # infrastructure adjustment rate (how quick infrastructure responds to adoption)
    network_type="BA",    # You can change this later for Part 2
    n_nodes=120,
    p=0.05, # edge probability (ER)
    m=2, # number of edges to attach for new nodes (BA)
)


# Parameter sweep X0 x ratio
This is the core baseline plot. Suggested exploratory resolution: X₀ 21 points, ratio 41 points.

How to interpret (for report):

Look for sharp color changes (blue→yellow) across X₀ at fixed ratio → tipping point.

Mixed-color vertical bands or “patchy” areas indicate bistability (same params give different outcomes across replicates).

The white dashed line X = 1 / ratio is theoretical threshold; compare model results to it.

In [3]:
X0_values = np.linspace(0, 1, 21) # Creates 21 evenly spaced initial adoption fractions from 0.0 to 1.0
ratio_values = np.linspace(0.8, 3.5, 41) # Creates 41 values for the payoff ratio a_I / b to sweep across plausible values.

phase_df = phase_sweep_df(
    X0_values = X0_values,
    ratio_values = ratio_values,
    scenario_kwargs = scenario_base,
    batch_size = 3,        # for computational efficiency
    init_noise_I = 0.04,
    T = 100, # for computational efficiency
    strategy_choice_func = "logit",
    tau = 1.0,
    max_workers = 1,
    backend = "thread"
)

path = plot_phase_plot(phase_df)
print("Saved phase plot to:", path)





Saved phase plot to: c:\Users\ecate\Desktop\Assignment 3\Assignment-3-MBDM\plots\ev_phase_plot.png


# Parameter sweep I0 x ratio

In [ ]:
I0_values = np.linspace(0, 1, 21) 
ratio_values = np.linspace(0.8, 3.5, 41) 
phase_df = phase_sweep_df(
    X0_values = I0_values,
    ratio_values = ratio_values,
    scenario_kwargs = scenario_base,
    batch_size = 3,        # for computational efficiency
    init_noise_I = 0.04,
    T = 100, # for computational efficiency
    strategy_choice_func = "logit",
    tau = 1.0,
    max_workers = 1,
    backend = "thread"
)

path = plot_phase_plot(phase_df)
print("Saved phase plot to:", path)


Saved phase plot to: c:\Users\ecate\Desktop\Assignment 3\Assignment-3-MBDM\plots\ev_phase_plot.png


# Parameter sweep beta0 x ratio

In [33]:
beta_I__values = np.linspace(0, 1, 21) 
ratio_values = np.linspace(0.8, 3.5, 41) 
phase_df = phase_sweep_df(
    X0_values = beta_I__values,
    ratio_values = ratio_values,
    scenario_kwargs = scenario_base,
    batch_size = 3,        # for computational efficiency
    init_noise_I = 0.04,
    T = 100, # for computational efficiency
    strategy_choice_func = "logit",
    tau = 1.0,
    max_workers = 1,
    backend = "thread"
)

path = plot_phase_plot(phase_df)
print("Saved phase plot to:", path)


Saved phase plot to: c:\Users\ecate\Desktop\Assignment 3\Assignment-3-MBDM\plots\ev_phase_plot.png


# Ratio sweep (fixed X0)
Pick typical X₀ values (e.g., 0.1, 0.2, 0.4). This shows 1D tipping curves. In the code below we run the sweeps for 3 X0 values to detect how sensitive tipping points are to initial EV adoption.

Interpretation: sudden vertical-like jumps on these plots indicate tipping points for that X₀.

Example wording for the report: "We performed ratio sweeps at multiple fixed initial adoption levels (X₀ = 0.10, 0.20, 0.40).
Each individual sweep keeps X₀ constant and varies the payoff ratio.
Comparing these sweeps reveals how the tipping point shifts with initial adoption."

In [30]:
for X0 in [0.40]: # initial values were 0.1,0.2,0.4
    print("Running ratio sweep for X0 =", X0)
    sweep_df = ratio_sweep_df(
        X0_frac = X0,
        ratio_values = np.linspace(0.8, 3.5, 41),
        scenario_kwargs = scenario_base,
        batch_size = 5,
        T = 150,
        strategy_choice_func = "logit",
        tau = 1.0
    )

fp = plot_ratio_sweep(sweep_df)
print("Saved ratio sweep:", fp)


Running ratio sweep for X0 = 0.4
Saved ratio sweep: c:\Users\ecate\Desktop\Assignment 3\Assignment-3-MBDM\plots\ev_ratio_sweep.png


# Ratio sweep (fixed I0)

In [25]:
#I0_list = [0.1, 0.2, 0.3]

#all_phase_plots = []

#for I0 in I0_list:
    #print("Running for I0 =", I0)
    
    #scenario_I0 = {**scenario_base, "I0": I0}
    
    #phase_df = phase_sweep_df(
        #X0_values=np.linspace(0,1,21),
        #ratio_values=np.linspace(0.8,3.5,41),
        #scenario_kwargs=scenario_I0
#    )
    
#plot_phase_plot(phase_df)

for I0 in [0.6]:
    print("Running for I0", I0)
    sweep_df = ratio_sweep_df(
        X0_frac=I0,
        ratio_values = np.linspace(0.8,3.5,41),
        scenario_kwargs=scenario_base,
        batch_size=3,
        T=100,
        strategy_choice_func="logit",
        tau=1
    )
fp = plot_ratio_sweep(sweep_df)
print("saved ratio sweep:", fp)

Running for I0 0.6
saved ratio sweep: c:\Users\ecate\Desktop\Assignment 3\Assignment-3-MBDM\plots\ev_ratio_sweep.png


# Ratio sweep (fixed beta)

In [36]:
for beta_i in [0.6]:
    print("Running for beta_i", beta_i)
    sweep_df = ratio_sweep_df(
        X0_frac=beta_i,
        ratio_values = np.linspace(0.8,3.5,41),
        scenario_kwargs=scenario_base,
        batch_size=3,
        T=100,
        strategy_choice_func="logit",
        tau=1
    )
fp = plot_ratio_sweep(sweep_df)
print("saved ratio sweep:", fp)

Running for beta_i 0.6
saved ratio sweep: c:\Users\ecate\Desktop\Assignment 3\Assignment-3-MBDM\plots\ev_ratio_sweep.png


# Show bistability and path dependence with spaghetti/denisty 
Pick a parameter setting inside the mixed zone from the phase plot (e.g., ratio ≈ 1.5, X₀ ≈ 0.3). Run collect_intervention_trials with no policy to collect many trajectories, then plot spaghetti/density.
Interpretation:

If many traces split to 0 or 1 from the same settings → bistability / path dependence. Use a histogram of final X(T) to show bimodality.

In [10]:
# Use actual values found from phase plot; these are examples
scenario_bist = {**scenario_base, "ratio": 2.5, "X0_frac": 0.40}
# collect_intervention_trials expects subsidy params but we can pass defaults and set n_trials
baseline_X, baseline_I, subs_X, subs_I, base_df, subs_df = collect_intervention_trials(
    n_trials = 80,    # 80 trials is good for visualizing distribution
    T = 100,
    scenario_kwargs = scenario_bist,
    subsidy_params = None,
    max_workers = 1,
    seed_base = 42
)

# baseline_X is list of X(t) arrays
traces_df = traces_to_long_df(baseline_X, baseline_X)  # trick: group both columns (we'll just view baseline)
# plot spaghetti and density using baseline as both groups: use plot_spaghetti and plot_density from ev_plotting
sp_path = plot_spaghetti(traces_df)
den_path = plot_density(traces_df, time_bins=200)
print("spaghetti:", sp_path, "density:", den_path)


spaghetti: c:\Users\ecate\Desktop\Assignment 3\Assignment-3-MBDM\plots\ev_spaghetti.png density: c:\Users\ecate\Desktop\Assignment 3\Assignment-3-MBDM\plots\ev_density.png


# Network structure comparison

Goal: run the same baseline sweeps for different network topologies and compare.

Supported networks in your code: "random" (Erdős–Rényi) and "BA" (Barabási–Albert). If you want a third (e.g., Watts–Strogatz), you can add it by implementing a small function that constructs a WS graph in ev_core.py and exposing it — but BA and random suffice for the assignment.

Do the same phase_sweep_df and ratio_sweep_df for each network type.

How to compare:

Place the images side-by-side in your report or presentation.

Note where the tipping boundary moves: networks with hubs (BA) often require lower X₀ to tip if hubs are seeded.

If you want to demonstrate seeding hubs vs random seeding: use set_initial_adopters(..., method="degree", high=True) when running run_timeseries_trial in a small loop — I can give exact code for targeted seeding if you want.


In [11]:
networks = ["random", "BA"]
for net in networks:
    print("Network:", net)
    scen = {**scenario_base, "network_type": net}
    phase_df_net = phase_sweep_df(
        X0_values = np.linspace(0.0, 1.0, 21),
        ratio_values = np.linspace(0.8, 3.5, 41),
        scenario_kwargs = scen,
        batch_size = 3,
        T = 100,
        strategy_choice_func = "logit",
        tau = 1.0,
        max_workers = 1,
        backend = "thread"
    )
    out = plot_phase_plot(phase_df_net)
    print("Saved:", out)


Network: random
Saved: c:\Users\ecate\Desktop\Assignment 3\Assignment-3-MBDM\plots\ev_phase_plot.png
Network: BA
Saved: c:\Users\ecate\Desktop\Assignment 3\Assignment-3-MBDM\plots\ev_phase_plot.png


# Policy intervention example

subsidy = dict(start=20, end=80, delta_a0=0.4)

baseline_X, baseline_I, subsidy_X, subsidy_I, base_df, subs_df = \
    collect_intervention_trials(
        n_trials=50,
        T=200,
        scenario_kwargs=scenario_base,
        subsidy_params=subsidy,
    )

traces_df = traces_to_long_df(baseline_X, subsidy_X)
plot_spaghetti(traces_df)
plot_density(traces_df)
